---
layout: post
title: Sprint5 Blog
description:  Leaderboard
type: issues 
comments: true
---

Group
The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.

Individual
My individual feature is primarily focused on making a leaderboard to showcase your scores. This feature allows people to see their highscores, low scores, and challenge themselves with default scores to earn a better score. The purpose of the feature is to also allow for a more gamelike experience for players.



## Fullstack


# Leaderboard API Technical Analysis

## 1. List Requests, Use of Lists, Dictionaries, and Database




### List Requests
- The `GET` method retrieves all leaderboard entries and returns them as a JSON response.

### Use of Lists and Dictionaries
- The `entries` variable is a list of `LeaderboardEntry` objects, and each entry is converted to a dictionary format for the JSON response.

### Formatting Response Data
- The API formats the results as a list of dictionaries, where each row in the database represents a list entry, and each column is stored as key-value pairs in a dictionary.

### Database Queries
- SQLAlchemy is used to query the database, returning Python lists representing rows of data.

### Methods in Class
- The `_CRUD` class within `LeaderboardAPI` defines methods to create, read, update, and delete entries, working with the columns of the database.

### Algorithmic Code Request
- The API class and methods are defined to handle different types of HTTP requests (`GET`, `POST`, `DELETE`). Each method in the class corresponds to a specific CRUD operation.



In [ ]:
# Import statements for Flask and required modules
from flask import Blueprint, request, g
from flask_restful import Api, Resource
from datetime import datetime
from __init__ import app
from api.jwt_authorize import token_required
from model.leaderboard import LeaderboardEntry
from model.competition import Time

# Create Blueprint for API routing - defines base URL prefix
leaderboard_api = Blueprint('leaderboard_api', __name__, url_prefix='/api')
api = Api(leaderboard_api)

class LeaderboardAPI:
    # CRUD Resource class handling all database operations
    class _CRUD(Resource):
        @token_required()  
        def post(self):
            """
            POST Method: Creates/updates leaderboard entries
            Parameters (JSON):
            - drawing_name: string
            - score: integer (optional)
            Returns: JSON response with entry data or error message
            """
            current_user = g.current_user
            data = request.get_json()  # Get JSON request body

            # Input validation example
            if not data or "drawing_name" not in data:
                return {
                    "message": "Missing drawing name",
                    "error": "Bad Request"
                }, 400

            try:
                # Database query using SQLAlchemy ORM
                competition_entry = Time.query.filter_by(
                    created_by=current_user.id,
                    drawn_word=data['drawing_name']
                ).first()

                # Algorithm for score calculation
                if "score" in data:
                    score = int(data['score'])
                else:
                    speed_factor = competition_entry.timer_duration / competition_entry.time_taken
                    score = min(1000, int(speed_factor * 500))

                # Database operation: Create or Update
                existing_entry = LeaderboardEntry.query.filter_by(
                    created_by=current_user.id,
                    drawing_name=data['drawing_name']
                ).first()

                if existing_entry:
                    # Update existing entry
                    if score > existing_entry.score:
                        existing_entry.score = score
                        existing_entry.update()
                    return existing_entry.read(), 200  # Returns JSON response
                else:
                    # Create new entry
                    entry = LeaderboardEntry(
                        profile_name=current_user.name,
                        drawing_name=data['drawing_name'],
                        score=score,
                        created_by=current_user.id
                    )
                    entry.create()
                    return entry.read(), 201  # Returns JSON response

            except Exception as e:
                return {
                    "message": "Failed to create/update entry",
                    "error": str(e)
                }, 500

        def get(self):
            """
            GET Method: Retrieves all leaderboard entries
            Demonstrates:
            - List operations (competition_entries list)
            - Dictionary operations (result dictionary)
            - Database queries (Time.query.all())
            - Iteration, Selection, and Sequencing
            Returns: JSON formatted leaderboard data
            """
            try:
                # Database query returning list of entries
                competition_entries = Time.query.all()  # SQLAlchemy query
                result = {}  # Dictionary to store results

                # Iteration through database rows
                for entry in competition_entries:
                    word = entry.drawn_word
                    # Dictionary manipulation
                    if word not in result:
                        result[word] = []

                    # Algorithm for score calculation
                    speed_factor = entry.timer_duration / entry.time_taken
                    score = min(1000, int(speed_factor * 500))

                    # Database operation with conditional logic
                    leaderboard_entry = LeaderboardEntry.query.filter_by(
                        created_by=entry.created_by,
                        drawing_name=word
                    ).first()

                    # Complex conditional logic
                    if leaderboard_entry and not leaderboard_entry.is_deleted:
                        if score > leaderboard_entry.score:
                            leaderboard_entry.score = score
                            leaderboard_entry.update()
                        result[word].append(leaderboard_entry.read())

                # Sort entries by score (list manipulation)
                for word in result:
                    result[word] = sorted(
                        result[word],
                        key=lambda x: x['score'],
                        reverse=True
                    )

                return result  # Returns JSON formatted data

            except Exception as e:
                print(f"Error in get method: {str(e)}")
                return {}

        @token_required()
        def delete(self):
            """
            DELETE Method: Removes leaderboard entries
            Parameters (JSON):
            - id: integer (entry ID to delete)
            Returns: JSON response with success/error message
            """
            current_user = g.current_user
            
            # Admin role validation
            if current_user.role != 'Admin':
                return {
                    "message": "Admin access required",
                    "error": "Forbidden"
                }, 403

            # Request validation
            data = request.get_json()
            if not data or "id" not in data:
                return {
                    "message": "Missing entry ID",
                    "error": "Bad Request"
                }, 400

            try:
                # Database query and update
                entry = LeaderboardEntry.query.get(data['id'])
                if not entry:
                    return {
                        "message": "Entry not found", 
                        "error": "Not Found"
                    }, 404

                entry.is_deleted = True
                entry.update()
                
                return {
                    "message": "Entry deleted successfully"
                }, 200

            except Exception as e:
                return {
                    "message": "Failed to delete entry",
                    "error": str(e)
                }, 500

    # Register endpoints with Flask-RESTful
    api.add_resource(_CRUD, '/leaderboard')

## 2. Parameters and Return Type
The body of the request contains JSON data with the following fields:

- profile_name: User's Id
- drawing_name: The name of the drawing.
- score: The score achieved.

#### Example IRL and JSON:
![image-2.png](https://i.ibb.co/1jV0NsY/image.png)

![image-2.png](https://i.ibb.co/tw8WkBQZ/image.png)

## 4. Call to Algorithm Request

### Call/Request to the Method
- The `fetchLeaderboard` function sends a `GET` request to the API endpoint to fetch leaderboard data.

### Return/Response from the Method
- The response is parsed as JSON and processed to update the DOM. Errors are logged and displayed in the DOM.

### Handling Normal and Error Conditions
- The `submitScore` and `deleteEntry` functions handle changing data and trigger different responses based on normal and error conditions, updating the DOM accordingly.


#### Call/Request Fullstack:

In [ ]:
# Import API configuration - Sets up base URL for API requests
import { pythonURI } from '{{site.baseurl}}/assets/js/api/config.js';

# Global admin flag
let isAdmin = false;

# Initial setup and admin check when page loads
document.addEventListener('DOMContentLoaded', async () => {
    try:
        # Algorithm Request: Fetch user data to verify admin status
        const response = await fetch(`${pythonURI}/api/user`, {
            credentials: 'include'
        });
        const userData = await response.json();
        isAdmin = userData.role === 'Admin';
        fetchLeaderboard();
    except error:
        # Error handling for failed admin check
        console.error('Error checking admin status:', error);
        showMessage('Error loading user data', 'error');
});

# Algorithm Request: Main function to fetch and display leaderboard data
async function fetchLeaderboard() {
    try:
        # Call/Request to leaderboard endpoint
        const response = await fetch(`${pythonURI}/api/leaderboard`, {
            credentials: 'include'
        });
        
        # Error condition validation
        if (!response.ok) throw new Error('Failed to fetch leaderboard data');
        
        # Return/Response processing
        const data = await response.json();
        const container = document.getElementById('leaderboard-sections');
        container.innerHTML = '';

        # Process returned data and update DOM with results
        Object.entries(data).forEach(([word, entries]) => {
            container.appendChild(createWordSection(word, entries));
        });
    except error:
        # Error condition handling for failed requests
        console.error('Error:', error);
        showMessage(error.message, 'error');
}

# Form submission handler with score validation and API request
document.getElementById('score-form').addEventListener('submit', async function(event) {
    event.preventDefault();
    const drawingName = document.getElementById('drawingName').value.trim();
    const score = document.getElementById('score').value;
    
    try:
        # Request data preparation and validation
        const body = { drawing_name: drawingName };
        if (score) {
            # Data validation - Normal vs Error condition
            const scoreNum = parseInt(score);
            if (scoreNum < 0 || scoreNum > 1000) {
                showMessage('Score must be between 0 and 1000', 'error');
                return;
            }
            body.score = scoreNum;
        }

        # Algorithm Request: POST score data to API
        const response = await fetch(`${pythonURI}/api/leaderboard`, {
            method: 'POST',
            credentials: 'include',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify(body)
        });

        # Response handling and validation
        const data = await response.json();
        
        # Condition handling for response
        if (!response.ok) throw new Error(data.message);
        
        # Normal condition: Success path
        showMessage('Score submitted successfully!', 'success');
        this.reset();
        await fetchLeaderboard();
    except error:
        # Error condition handling
        console.error('Error:', error);
        showMessage(error.message, 'error');
});

# Delete entry function with admin validation and API request
window.deleteEntry = async function(id) {
    # Pre-request validation for admin access
    if (!isAdmin) {
        # Error condition: Unauthorized access
        showMessage('Admin access required', 'error');
        return;
    }

    # User confirmation before deletion
    if (!confirm('Are you sure you want to delete this entry?')) return;
    
    try:
        # Algorithm Request: DELETE API call
        const response = await fetch(`${pythonURI}/api/leaderboard`, {
            method: 'DELETE',
            credentials: 'include',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ id })
        });

        # Response handling and validation
        const data = await response.json();
        
        # Condition handling for response
        if (!response.ok) throw new Error(data.message);
        
        # Normal condition: Success path
        showMessage('Entry deleted successfully', 'success');
        await fetchLeaderboard();
    except error:
        # Error condition handling
        console.error('Error:', error);
        showMessage(error.message, 'error');
};

# Helper function for displaying timed messages with different styles
function showMessage(text, type) {
    const messageEl = document.getElementById('message');
    messageEl.textContent = text;
    messageEl.className = `message ${type}`;
    messageEl.style.display = 'block';
    
    # Auto-hide message after delay using timeouts
    setTimeout(() => {
        messageEl.style.opacity = '0';
        setTimeout(() => {
            messageEl.style.display = 'none';
            messageEl.style.opacity = '1';
        }, 300);
    }, 3000);
}